In [ ]:
# Install required libraries
!pip install googletrans==4.0.0-rc1 gTTS ipywidgets

from googletrans import Translator
from gtts import gTTS
import IPython.display as display
import ipywidgets as widgets
import tempfile, os

# Initialize translator
translator = Translator()

# Language options (common ones)
languages = {
    'English': 'en',
    'Spanish': 'es',
    'French': 'fr',
    'German': 'de',
    'Chinese (Simplified)': 'zh-cn',
    'Hindi': 'hi',
    'Arabic': 'ar',
    'Russian': 'ru',
    'Japanese': 'ja',
    'Korean': 'ko',
}

# Widgets
input_text = widgets.Textarea(
    placeholder='Enter text to translate...',
    layout=widgets.Layout(width='100%', height='100px')
)

source_lang = widgets.Dropdown(
    options=languages.keys(),
    value='English',
    description='Source:'
)

target_lang = widgets.Dropdown(
    options=languages.keys(),
    value='Spanish',
    description='Target:'
)

translate_button = widgets.Button(description="Translate", button_style='success')
output_area = widgets.Output()

copy_button = widgets.Button(description="Copy Result", button_style='info')
voice_button = widgets.Button(description="Speak", button_style='warning')

translated_text_global = ""  # to store last translation

# Translation function
def translate_text(b):
    global translated_text_global
    with output_area:
        output_area.clear_output()
        try:
            result = translator.translate(input_text.value, src=languages[source_lang.value], dest=languages[target_lang.value])
            translated_text_global = result.text
            print("Translated Text:")
            print(result.text)
        except Exception as e:
            print("Error:", e)

# Copy to clipboard (only works in notebook display)
def copy_result(b):
    if translated_text_global:
        js_code = f'''
        navigator.clipboard.writeText("{translated_text_global}");
        alert("Copied to clipboard!");
        '''
        display.display(display.Javascript(js_code))

# Text-to-Speech
def speak_result(b):
    if translated_text_global:
        tts = gTTS(translated_text_global, lang=languages[target_lang.value])
        tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(tmpfile.name)
        display.display(display.Audio(tmpfile.name, autoplay=True))

# Button bindings
translate_button.on_click(translate_text)
copy_button.on_click(copy_result)
voice_button.on_click(speak_result)

# UI Layout
display.display(widgets.VBox([
    input_text,
    widgets.HBox([source_lang, target_lang]),
    translate_button,
    output_area,
    widgets.HBox([copy_button, voice_button])
]))